# BBC - Projet
## Diagnostic miARN pour détecter le cancer de la prostate

- Professeur: Carlos Peña (<a href="mailto:carlos.pena@heig-vd.ch">carlos.pena@heig-vd.ch</a>)
- Assistante: Aitana Lebrand (<a href="mailto:aitana.nevesdasilva@heig-vd.ch">aitana.nevesdasilva@heig-vd.ch</a>)
- Etudiant: Jan Purro (<a href="mailto:jan.purro@heig-vd.ch">jan.purro@heig-vd.ch</a>)
- Etudiant: Thibault Schowing (<a href="mailto:thibault.schowing@heig-vd.ch">thibault.schowing@heig-vd.ch</a>)

Date: Printemps 2016

> ## Objectifs pédagogiques


> ## But du projet



> ## Démarche


> ## Rapport de labo



In [1]:
import os
import numpy as np
import pylab as plt
import pandas as pd
import numpy as np
from Bio import Geo

def load_geo(myfile):
    handle = open(myfile)
    records = Geo.parse(handle)
    return records

# Les différents records, le soft n'es pas forcément utile.
#records_soft = load_geo('../data/GSE45604_family.soft')
records_matrix = open('../data/GSE45604_series_matrix.txt')

# Données brutes (uniquement les lignes avec condition et miARN)
matrice =  [l.split('\t') for l in records_matrix.readlines()[67:-1]]
records_matrix.seek(0)
conditions = records_matrix.readlines()[34].split('\t')

print(type(matrice))


matrice = np.array(matrice)
conditions = np.array(conditions[1:])

# On a 2 types de cellules: prostate normale ou cancéreuse qu'on va marquer PCa ou Normal
i = 1
for c in conditions:
    if c.startswith('"PCa'):
        matrice[0,i] = 'PCa'
    else:
        matrice[0,i] = 'Normal'
    i += 1

# TODO: mettre dans l'ordre PCa puis Normale

# AVANT TRANSPOSITION
# Matrice[0] contient ID_REF - PCa - PCa - ... - Normal - PCa - ...
# Matrice[1] contient 234342_st - 23.3 - 342.3 - ...
# Matrice[:,0] contient ID_REF suivi de tous les id de gènes
# Matrice[:,1...n] Contient PCa ou normal suivi de toutes les mesures pour ce prélèvement sur un patient atteint ou non

cell_type = ['Normal','PCa']


matrice = matrice.transpose()
#print(matrice[1:,0])


# Matrice[0] Contient maintenant: ID_REF - 1234_st - 3456_st - ... 
# Matrice[1] Contient maintenant: Normal - 123.2   -  23.2   - ...
# Matrice[1:,0] Contient les types de cellules (PCa ou Normal) dans l'ordre après le sort
# 
matrice.sort(axis=0)

# Les gènes prélèvement sont triés Normal - PCa
#print(matrice[1:,0])

<type 'list'>


In [ ]:
# On retranspose la matrice 

matrice = matrice.transpose()


In [2]:
# Diviser le dataset en deux -> test et train


print matrice[0,:]
print(np.shape(matrice))

idx_test = [5,11,19,28,32,60]
idx_train = range(0,len(matrice[0]))
[idx_train.remove(i) for i in idx_test]

print("IDX test: ", idx_test)
print("IDX train: ", idx_train)

data = matrice
data_test = data[:,idx_test]
data = data[:,idx_train]

print(matrice)


['"ID_REF"' 'Normal' 'Normal' 'Normal' 'Normal' 'Normal' 'Normal' 'Normal'
 'Normal' 'Normal' 'Normal' 'PCa' 'PCa' 'PCa' 'PCa' 'PCa' 'PCa' 'PCa' 'PCa'
 'PCa' 'PCa' 'PCa' 'PCa' 'PCa' 'PCa' 'PCa' 'PCa' 'PCa' 'PCa' 'PCa' 'PCa'
 'PCa' 'PCa' 'PCa' 'PCa' 'PCa' 'PCa' 'PCa' 'PCa' 'PCa' 'PCa' 'PCa' 'PCa'
 'PCa' 'PCa' 'PCa' 'PCa' 'PCa' 'PCa' 'PCa' 'PCa' 'PCa' 'PCa' 'PCa' 'PCa'
 'PCa' 'PCa' 'PCa' 'PCa' 'PCa' 'PCa']
(20644, 61)
('IDX test: ', [5, 11, 19, 28, 32, 60])
('IDX train: ', [0, 1, 2, 3, 4, 6, 7, 8, 9, 10, 12, 13, 14, 15, 16, 17, 18, 20, 21, 22, 23, 24, 25, 26, 27, 29, 30, 31, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59])
[['"ID_REF"' 'Normal' 'Normal' ..., 'PCa' 'PCa' 'PCa']
 ['"1367452_st"' '10.5609' '10.93488' ..., '20.23703' '8.428252' '9.889372']
 ['"1367453_st"' '12.8113' '12.84643' ..., '21.90498' '22.80263'
  '25.17342\n']
 ..., 
 ['"zma-miR529_st"' '11.23052' '11.74796' ..., '20.07965' '20.20208'
  '20.75771']
 ['"zma-

In [3]:
# On va mettre les datas dans l'ordre

#print(list(matrice[0,1:]))
print(data)

cell_type = 


SyntaxError: invalid syntax (<ipython-input-3-693aff47c674>, line 6)